# DS-SF-27 | Codealong 17 | Introduction to Databases

## RMDBS databases and `SQLite`

In [ ]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [ ]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

### `SELECT`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

### `WHERE`

In [ ]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price
    FROM transactions
        WHERE sale_price_unit = '$'
;
''', con = db)

> #### Here's let's convert the unit of `sale_price` from \$M to \$

In [ ]:
pd.io.sql.read_sql('''
SELECT ID, sale_price * 1000000 AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$ accross the dataset

In [ ]:
pd.io.sql.read_sql('''
SELECT ID, sale_price
    FROM transactions
        WHERE sale_price_unit = '$'
UNION ALL
SELECT ID, sale_price * 1000000 AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

### `WITH`

In [ ]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

### `JOIN`

In [ ]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
    FROM properties
        JOIN normalized_sale_prices ON normalized_sale_prices.ID = properties.ID
;
''', con = db)

### `WITH` (take 2)

In [ ]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

bed_count_normalized_sale_prices AS
    (SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT * FROM bed_count_normalized_sale_prices
;
''', con = db)

### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [ ]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

bed_count_normalized_sale_prices AS
    (SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT AVG(sale_price) AS mean_sale_price
    FROM bed_count_normalized_sale_prices
        GROUP BY bed_count
;
''', con = db)

Note: `bed_count = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [ ]:
# TODO

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [ ]:
# TODO

## NoSQL databases and `CouchDB`

### Administration Web Interface

> `CouchDB`'s Administration Web Interface, called `Fauxton` is at http://localhost:5984/_utils

### A brief overview of `CouchDB`'s bare-bone API using the command-line utility `curl`

> #### Hello World!

In [ ]:
!curl -X GET http://localhost:5984

> #### Get the list of databases

In [ ]:
!curl -X GET http://localhost:5984/_all_dbs

> ### Create a new database called `zillow`

In [ ]:
!curl -X PUT http://localhost:5984/zillow

> ### Check out the new database

In [ ]:
!curl -X GET http://localhost:5984/zillow

It's a new database so it is empty (`"doc_count":0`).  Let's now populate it.

> ### Bulk import

In [ ]:
!curl -d @../datasets/zillow_pretty.json -H "Content-Type: application/json" -X POST http://localhost:5984/zillow/_bulk_docs

Let's check it again...

In [ ]:
!curl -X GET http://localhost:5984/zillow

We can see that we now have 1,000 documents in it (`"doc_count":1000`).  Let's have a look at a couple of documents.

> ### `_id`

In [ ]:
!curl -X GET http://localhost:5984/zillow/15083161

That's our very expensive \$32M home!  15083161 refers to the Zillow identifier of our dataset but how did `CouchDB` know how to use it?  For the bulk import, we set the identifier of each document with `_id`.

In [ ]:
!curl -X GET http://localhost:5984/zillow/15149005

Our \$1 home...

## List of all documents within a database

In [ ]:
!curl -X GET http://localhost:5984/zillow/_all_docs

> Now with the document contents

In [ ]:
!curl -X GET http://localhost:5984/zillow/_all_docs?include_docs=true

### Let's now using `CouchDB`'s `MapReduce` to compute the average sale price of these properties based on their number of bedrooms

> Using `Fauxton`, let's create a new view: http://localhost:5984/_utils/#/database/zillow/new_view

Copy/paste the following (JavaScript) code for the `Map function`.

```javascript
function(property) {
    if (property.bed_count) {
        bed_count = property.bed_count;
    } else if (property.is_a_studio == true) {
        bed_count = 0;
    } else {
        return;
    }
    if (property.sale.price.unit == '$') {
        emit(bed_count, [property.sale.price.value, 1]);
        return;
    }
    if (property.sale.price.unit == '$M') {
        emit(bed_count, [property.sale.price.value * Math.pow(10, 6), 1]);
        return;
    }
}
```

Copy/paste the following (JavaScript) code for the `Map function` (chose `CUSTOM`).

```javascript
function(keys, values) {
    weighted_price = 0
    weight = 0
        values.forEach(function (value) {
            weighted_price += value[0] * value[1];
            weight += value[1];
        });
     return [weighted_price / weight, weight];
}
```

Click on `Options` (upper right), check `Reduce` and `Run Query`.  Do these numbers match what we got from the `SQL` section?

### Delete a database

In [ ]:
!curl -X DELETE http://localhost:5984/zillow